# Example using Tensorboard and a simple model

In [ ]:
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Rescaling
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import TensorBoard
from pathlib import Path
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd

## Settings

In [ ]:
batch_size = 2
img_size = 64
epochs = 20
seed = 27
validation_split = 0.2

wd = Path('../../..')
dataset_path = Path(wd, 'data/dataset_example')
train_path = Path(dataset_path, "train")
test_path = Path(dataset_path, "test")
model_path =  Path(wd, "models/example_cnn.h5")
plots_path = Path(wd, "plots")
log_dir = Path(wd, "logs", datetime.now().strftime("%Y%m%d-%H%M%S"))

class_names = ['amanita', 'boletus']

## Load data

In [ ]:
training_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='training',
    labels='inferred',
    class_names=class_names,
    label_mode='binary',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

validation_data = image_dataset_from_directory(
    directory=train_path,
    validation_split=validation_split,
    subset='validation',
    labels='inferred',
    class_names=class_names,
    label_mode='binary',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

test_data = image_dataset_from_directory(
    directory=test_path,
    labels='inferred',
    class_names=class_names,
    label_mode='binary',
    batch_size=batch_size,
    image_size=(img_size, img_size),
    seed=seed,
    shuffle=True
)

## Define model

In [ ]:
model = Sequential([
    Rescaling(
        scale=1./255,
        input_shape=(img_size, img_size, 3)
    ),
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding='same',
        activation='relu'
    ),
    MaxPooling2D(
        pool_size=(2, 2),
        strides=(2, 2)
    ),
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        padding='same',
        activation='relu'
    ),
    MaxPooling2D(
        pool_size=(2, 2),
        strides=(2, 2)
    ),
    Conv2D(
        filters=256,
        kernel_size=(3, 3),
        padding='same',
        activation='relu'
    ),
    MaxPooling2D(
        pool_size=(2, 2),
        strides=(2, 2)
    ),
    Flatten(),
    Dense(
        units=2048,
        activation='relu'
    ),
    Dense(
        units=512,
        activation='relu'
    ),
    Dense(
        units=1,
        activation='sigmoid'
    )
])

## Compile model

In [ ]:
model.compile(
    optimizer=SGD(),
    loss=BinaryCrossentropy(),
    metrics=['accuracy']
)

## Define Tensorboard callback

In [ ]:
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1
)

## Train model

In [ ]:
history = model.fit(
    training_data,
    validation_data=validation_data,
    epochs=epochs,
    callbacks=[tensorboard_callback]
)

## Save model

In [ ]:
model.save(model_path)

## Accuracy and loss plot for training and validation

In [ ]:
metrics_df = pd.DataFrame(history.history)

In [ ]:
metrics_df[["accuracy", "val_accuracy"]].plot()
plt.title('Training and Validation Accuracy')
plt.savefig(Path(plots_path, 'accuracy.png'))
plt.show()

In [ ]:
metrics_df[["loss", "val_loss"]].plot()
plt.title('Training and Validation Loss')
plt.savefig(Path(plots_path, 'loss.png'))
plt.show()

## Evaluate model

In [ ]:
model.evaluate(test_data)